### **Draft con primeros pasos ETL dataset [Digital Nomad Index Ranked Cicleloop](https://www.circleloop.com/nomadindex/)**

1. Extracción datos raw del csv Digital Nomad Index Ranked Cicleloop. 
2. Análisis explotario superficial de variables, volumen de nulos por variable y datos estadísticos generales. 
3. Limpieza de encodeados de algunas columnas que contienen errores. 
4. Limpieza y normalización de naming de algunas columnas. 
5. Creación de una función de conversión a dólares para tener todos los dataset en esa moneda previa al EDA. 
6. Creación de un archivo CSV nuevo con el dataset 100% limpio. 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

pd.options.mode.copy_on_write = True # CoW por defecto a partir de pandas 3.0.0 

In [2]:
# Extraigo y examino algunos de los datos que devuelve el dataset.
# Uso encoding latin-1 porque si no da problemas al abrir el archivo
# Instacionamos el dataframe para empezar a analizarlo 

df_nomad_index = pd.read_csv("./data/digital-nomad_index_ranked_circleloop.csv", encoding = "latin-1")
df_nomad_index

,#,Country,Broadband SpeedAverage Fixed Broadband Speed (Mbps),Mobile Speed,Broadband Cost,Monthly Rent,Working Holidays Visa,Happiness Index,Migrant Population %,Remote Jobs Searches,Digital Nomad Score
0,1,Canada,149.35,84.54,32.42,"950,00 ",NaN,7.23,21.3,83900,74.35
1,2,UK,76.49,41.72,33.21,"927,00 ",NaN,7.17,14.1,68400,63.43
2,3,Romania,188.55,41.48,7.58,"325,00 ",NaN,6.12,2.4,10980,62.28
3,4,Sweden,158.73,56.64,36.83,"885,00 ",NaN,7.35,20.0,3490,61.54
4,5,Denmark,179.81,66.68,44.85,"1.059,00 ",NaN,7.65,12.5,1080,61.49
...,...,...,...,...,...,...,...,...,...,...,...
80,81,Sri Lanka,27.57,17.04,25.48,"278,00 ",NaN,4.33,0.2,230,32.08
81,82,Kenya,17.66,22.47,52.18,"262,00 ",NaN,4.58,2.0,3060,31.27
82,83,Ghana,47.46,15.04,64.67,"591,00 ",NaN,5.15,1.5,80,30.36
83,84,Myanmar,19.13,22.83,34.24,"665,00 ",NaN,4.31,0.1,150,29.16


#### **2. Análisis exploratorio de variables del Dataset**

Visualizamos algunos datos generales del dataset (especificidad, nulos y datos estadísticos generales)

In [ ]:
df_nomad_index.head(20)

# Aquí podemos ver algunos problemas de encodig en variables como 'Broadband Cost' o 'Monthly Rent'.
# Y vemos que 'Working Holidays Visa' tiene muchos nulos. 

,#,Country,Broadband SpeedAverage Fixed Broadband Speed (Mbps),Mobile Speed,Broadband Cost,Monthly Rent,Working Holidays Visa,Happiness Index,Migrant Population %,Remote Jobs Searches,Digital Nomad Score
0,1,Canada,149.35,84.54,32.42,"950,00 ",NaN,7.23,21.3,83900,74.35
1,2,UK,76.49,41.72,33.21,"927,00 ",NaN,7.17,14.1,68400,63.43
2,3,Romania,188.55,41.48,7.58,"325,00 ",NaN,6.12,2.4,10980,62.28
3,4,Sweden,158.73,56.64,36.83,"885,00 ",NaN,7.35,20.0,3490,61.54
4,5,Denmark,179.81,66.68,44.85,"1.059,00 ",NaN,7.65,12.5,1080,61.49
5,6,France,177.93,50.45,25.86,"758,00 ",NaN,6.66,12.8,5360,60.80
6,7,Netherlands,125.82,88.13,39.46,"1.213,00 ",NaN,7.45,13.4,3440,60.27
7,8,Australia,58.52,88.35,44.97,"1.148,00 ",NaN,7.22,30.0,17600,60.16
8,9,Switzerland,186.40,73.85,67.05,"1.545,00 ",NaN,7.56,29.9,3840,60.15
9,10,Germany,120.13,49.67,26.73,"824,00 ",NaN,7.08,15.7,12720,60.00


In [ ]:
df_nomad_index.info()

# Aquí ya podemos ver que existe la variable 'Working Holidays Visa' tiene todos los valores nulos. 
# Tendremos que eliminarla del dataset. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 11 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   #                                                    85 non-null     int64  
 1   Country                                              85 non-null     object 
 2   Broadband SpeedAverage Fixed Broadband Speed (Mbps)  85 non-null     float64
 3   Mobile Speed                                         85 non-null     float64
 4   Broadband Cost                                       85 non-null     object 
 5   Monthly Rent                                         85 non-null     object 
 6   Working Holidays Visa                                0 non-null      float64
 7   Happiness Index                                      85 non-null     float64
 8   Migrant Population %                                 85 non-null     flo

In [5]:
# Hacemos un sum de valores nulos para certificar que 'Working Holidays Visa' es una variable sin valor para el EDA. 

df_nomad_index.isnull().sum() 

#                                                       0
Country                                                 0
Broadband SpeedAverage Fixed Broadband Speed (Mbps)     0
Mobile Speed                                            0
Broadband Cost                                          0
Monthly Rent                                            0
Working Holidays Visa                                  85
Happiness Index                                         0
Migrant Population %                                    0
Remote Jobs Searches                                    0
Digital Nomad Score                                     0
dtype: int64

In [6]:
# Aquí vemos algunos datos estadísticos generales del dataset.

df_nomad_index.describe()

,#,Broadband SpeedAverage Fixed Broadband Speed (Mbps),Mobile Speed,Working Holidays Visa,Happiness Index,Migrant Population %,Remote Jobs Searches,Digital Nomad Score
count,85.000000,85.000000,85.000000,0.0,85.000000,85.000000,85.000000,85.000000
mean,43.000000,81.160706,42.682118,NaN,6.055647,11.510588,4741.647059,49.118824
std,24.681302,56.411813,23.164497,NaN,0.891339,15.386574,12279.242051,10.099860
min,1.000000,4.970000,12.680000,NaN,4.150000,0.100000,40.000000,26.320000
25%,22.000000,31.890000,24.630000,NaN,5.510000,1.700000,430.000000,38.970000
50%,43.000000,68.900000,38.880000,NaN,6.120000,5.300000,1050.000000,52.080000
75%,64.000000,120.130000,53.890000,NaN,6.660000,15.400000,3490.000000,56.570000
max,85.000000,229.420000,129.610000,NaN,7.810000,87.900000,83900.000000,74.350000


#### **3. Eliminación de la variable `Working Holidays Visa`**

In [10]:
# Eliminamos la columna Working Holidays Visa porque el 100% de sus valores son nulos

df_nomad_index = df_nomad_index.drop(columns=["Working Holidays Visa"], axis=1)

KeyError: "['Working Holidays Visa'] not found in axis"

In [11]:
df_nomad_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 10 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   #                                                    85 non-null     int64  
 1   Country                                              85 non-null     object 
 2   Broadband SpeedAverage Fixed Broadband Speed (Mbps)  85 non-null     float64
 3   Mobile Speed                                         85 non-null     float64
 4   Broadband Cost                                       85 non-null     object 
 5   Monthly Rent                                         85 non-null     object 
 6   Happiness Index                                      85 non-null     float64
 7   Migrant Population %                                 85 non-null     float64
 8   Remote Jobs Searches                                 85 non-null     int